
## **In this practice session, we will learn how to implement Recurrent Neural Networks for Time Series Forecasting**
## **We will use the stock prices of tesla dataset for the implementation**

**Data processing**

*   Import the required libraries from Keras
*   Upload the data to the notebook
*   Split the data into train and test set
*   Scale the training data between 0 and 1
*   Create a data structure with 60 time-steps and 1 output


**Build an RNN model**
* Construct a simple LSTM model
* Compile the model and fit the data into the model
* Preprocess the test data and make predictions on test data
* Plot the model predictions vs the actual data

## **Import the libraries needed**

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn keras tensorflow opencv-python scikit-image nltk bs4 wordcloud --user -q

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
#load the important libraries 
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

## **Display the dataset**

In [ ]:
#read and display the dataset
stock_price = pd.read_csv("TSLA.csv")
print(stock_price.shape)
stock_price.head()

## **Split the dataset into train and test**

In [ ]:
#split the dataset into training and test set
training_set = stock_price.iloc[:800, 1:3].values
test_set = stock_price.iloc[800:, 1:3].values

In [ ]:
print(training_set.shape, test_set.shape)

## **Scale the training data**

In [ ]:
#scaling the data between 0 and 1
scaling = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = scaling.fit_transform(training_set[:, 0].reshape(-1,1))

## **Creating a data structure with 60 time-steps and 1 output**

In [ ]:
# Creating a data structure with 60 time-steps and 1 output
trainX = []
trainY = []

for i in range(60, 800):
    trainX.append(training_set_scaled[i-60:i])
    trainY.append(training_set_scaled[i])
    
trainX, trainY = np.array(trainX), np.array(trainY)
# trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))

In [ ]:
print(trainX.shape, trainY.shape)

## **Build an LSTM model with dropout**

In [ ]:
#build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (trainX.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.summary()

## **Compile the model and begin training**

In [ ]:
#compile the model
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(trainX, trainY, nb_epoch=10, batch_size = 32)

## **Scale and pre-process test data**

In [ ]:
# Getting the predicted stock price of 2018
dataset_train = stock_price.iloc[:800, 1:3]
dataset_test = stock_price.iloc[800:, 1:3]

dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total.loc[len(dataset_total) - len(dataset_test) - 60:, :].values
inputs = inputs.reshape(-1,1)
inputs = scaling.transform(inputs)

X_test = []

for i in range(60, 519):
    X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print(X_test.shape)

## **Make model predictions on test data**

In [ ]:
#make the predictions
predicted_stock_price = model.predict(X_test)
# predicted_stock_price = np.reshape(X_test.shape[0], 1, 1)

predicted_stock_price = scaling.inverse_transform(predicted_stock_price)

## **Plot the model predictions**

In [ ]:
#plot the predictions
plt.plot(stock_price.loc[800:, 'Date'],dataset_test.values, color = 'red', label = 'Real TESLA Stock Price')
plt.plot(stock_price.loc[800:, 'Date'],predicted_stock_price, color = 'blue', label = 'Predicted TESLA Stock Price')
plt.xticks(np.arange(0,459,50))
plt.title('TESLA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TESLA Stock Price')
plt.legend()
plt.show()